## Зачет по ИИ

**Задание**: Построить классификатор для определения эмоции человека на фотографии.

In [ ]:
%%bash
wget -q https://dl.dropboxusercontent.com/s/hptn1l9jki7ie5v/emosubset.zip
mkdir emos
cd emos
unzip -q ../emosubset.zip

Данный датасет содержит изображения двух классов: `angry` и `surprise`. Разбейте датасет на обучающую и тестовую выборку сбалансированным обазом и постройте модель определения эмоции. Постройте матрицу ошибок на тестовой выборке чтобы посмотреть, как работет модель.

Необходимо добиться точности >75% на тестовой выборке.

Удачи!

In [ ]:
import tensorflow as tf
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.applications import VGG16

In [ ]:
data_dir = './emos'
train_dir = 'D:/emotrain1'
test_dir = 'D:/emotest1'

In [ ]:
classes = ['angry', 'surprise']

In [ ]:
for class_name in classes:
  class_dir = os.path.join(data_dir, class_name)
  train_class_dir = os.path.join(train_dir, class_name)
  test_class_dir = os.path.join(test_dir, class_name)
  os.makedirs(train_class_dir, exist_ok=True)
  os.makedirs(test_class_dir, exist_ok=True)
  image_files = os.listdir(class_dir)
  random.shuffle(image_files)
  split_index = int(0.8*len(image_files))
  train_files = image_files[:split_index]
  test_files = image_files[split_index:]

  for train_file in train_files:
    src_path = os.path.join(class_dir, train_file)
    dst_path = os.path.join(train_class_dir, train_file)
    shutil.copyfile(src_path, dst_path)

  for test_file in test_files:
    src_path = os.path.join(class_dir, test_file)
    dst_path = os.path.join(test_class_dir, test_file)
    shutil.copyfile(src_path, dst_path)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150),batch_size=32,class_mode='binary')

Found 525 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150),batch_size=32,class_mode='binary')

Found 257 images belonging to 2 classes.


In [ ]:
from keras.api._v2.keras import activations
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2,2)),
    Conv2D(16, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(8, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_90 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_84 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_91 (Conv2D)          (None, 72, 72, 16)        4624      
                                                                 
 max_pooling2d_85 (MaxPoolin  (None, 36, 36, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_92 (Conv2D)          (None, 34, 34, 8)         1160      
                                                                 
 max_pooling2d_86 (MaxPoolin  (None, 17, 17, 8)      

In [ ]:
model.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
17/17 [==============================] - 12s 594ms/step - loss: 0.6783 - accuracy: 0.5981 - val_loss: 0.6540 - val_accuracy: 0.6187
Epoch 2/10
17/17 [==============================] - 7s 410ms/step - loss: 0.6620 - accuracy: 0.6095 - val_loss: 0.6569 - val_accuracy: 0.6187
Epoch 3/10
17/17 [==============================] - 6s 384ms/step - loss: 0.6560 - accuracy: 0.6095 - val_loss: 0.6388 - val_accuracy: 0.6226
Epoch 4/10
17/17 [==============================] - 8s 464ms/step - loss: 0.6413 - accuracy: 0.6343 - val_loss: 0.6150 - val_accuracy: 0.6226
Epoch 5/10
17/17 [==============================] - 6s 387ms/step - loss: 0.6200 - accuracy: 0.6571 - val_loss: 0.5787 - val_accuracy: 0.6770
Epoch 6/10
17/17 [==============================] - 7s 444ms/step - loss: 0.5897 - accuracy: 0.6781 - val_loss: 0.5389 - val_accuracy: 0.7082
Epoch 7/10
17/17 [==============================] - 9s 567ms/step - loss: 0.5559 - accuracy: 0.7314 - val_loss: 0.5264 - val_accuracy: 0.7782
Epoch